In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep

import pandas as pd
import re
from datetime import datetime
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = 'https://incamp.ru/camps/education/2/'
response = requests.get(url, headers=headers)
response.text
soup = BeautifulSoup(response.text, 'lxml')
data = soup.find_all('div', class_='camp-item item')

In [6]:

camps_list = []

for part in data:
    # Название лагеря
    name_tag = part.find('div', class_='float-left title').find('a')
    name = name_tag.text.strip() if name_tag else None

    # Ссылка на лагерь
    link = 'https://incamp.ru' + name_tag['href'] if name_tag else None

    # Полное описание места (объединённые город и место)
    location = None
    location_div = part.find('div', class_='float-left title').find('div')
    if location_div:
        # Получаем чистый текст без HTML-тегов
        location = location_div.get_text(strip=True)
        # Удаляем ненужные префиксы
        location = location.replace('Россия, ', '').replace('РФ, ', '').strip()

    # Список тегов
    tags_container = part.find('div', class_='keywords clearfix')
    list_tags = [a.text.strip() for a in tags_container.find_all('a')] if tags_container else []

    # Рейтинг
    rating_block = part.find('div', class_='float-right rating')
    rating = None
    if rating_block:
        rating_span = rating_block.find('span', class_='rating_score')
        if rating_span:
            rating_text = rating_span.text.replace(',', '.').split()[0]
            try:
                rating = float(rating_text)
            except ValueError:
                pass

    # Собираем данные
    camps_list.append({
        'name': name,
        'link': link,
        'location': location,  # единое поле с описанием места
        'list_tags': list_tags,
        'rating': rating
    })

# Создаем DataFrame
df = pd.DataFrame(camps_list)

In [7]:
df

,name,link,location,list_tags,rating
0,Компьютерная Академия TOP. Котельники,https://incamp.ru/camp/top-kotelniki12770/,"Москва, ул. Привольная 70, БЦ Жулебино,Юго-Вос...","[Городской клуб, Кешбэк от incamp, Робототехни...",5.0
1,Компьютерная Академия TOP. Академическая,https://incamp.ru/camp/top-akademicheskaya-12768/,"Профсоюзная дом 3 , офис 222,Юго-Западный окру...","[Городской клуб, Кешбэк от incamp, Робототехни...",4.8
2,Каникулы будущего (м. ЦСКА),https://incamp.ru/camp/kanikuly-budushhego-cska/,"Москва, улица Гризодубовой, дом 4, корпус 3,Се...","[Городской клуб, Кешбэк от incamp, Творческий,...",5.0
3,Чуланчик,https://incamp.ru/camp/chulanchik-12943/,"г Москва, Озерковская наб, д 50 стр 1,Централь...","[Городской клуб, Кешбэк от incamp, Языковой, Т...",5.0
4,Дети вТеме,https://incamp.ru/camp/gorodskoy-dnevnoy-lager...,"г Москва, ул Вавилова, д 2,Центральный округ,м...","[Городской клуб, Кешбэк от incamp, Языковой, Т...",4.8
5,Инжиниум. ИТ каникулы в Сколково,https://incamp.ru/camp/inzhinium-klub-nauchno-...,"г Москва, тер инновационного центра Сколково, ...","[Городской клуб, Кешбэк от incamp, Образовател...",5.0
6,Алгоритм,https://incamp.ru/camp/culture-camp-algoritm-1...,"РФ,Москва и Московская обл.,Центральный округ,...","[Образовательные курсы и выезды, Кешбэк от inc...",NaN
7,Лингвамания,https://incamp.ru/camp/lingvamaniya-13034/,"РФ,Москва и Московская обл.,Ленинский район,м....","[Детский тур, Кешбэк от incamp, Языковой, , Об...",NaN
8,​Водное приключение,https://incamp.ru/camp/vodnoe-priklyuchenie-14...,"РФ,Москва и Московская обл.,Истринский районпа...","[Палаточный лагерь, Кешбэк от incamp, Оздорови...",NaN
9,Английский клуб Про,https://incamp.ru/camp/angliyskiy-klub-pro-13224/,"РФ,Москва и Московская обл.,Щелковский районпа...","[Детский тур, Кешбэк от incamp, Языковой, Тема...",NaN
